In [1]:
import sqlite3
import pandas as pd
import json
import requests
import os
import time
from pathlib import Path
from jinja2 import Environment, FileSystemLoader

## Création BDD et tables

In [2]:
try:
    db = sqlite3.connect("foil.db")
    
    cursor = db.cursor()
    
    cursor.execute('''create table stations (       
    
    id_station integer primary key,
    name varchar,
    latitude float,
    longitude float)''')
    
    cursor.execute('''create table mesures (       
    
    id_mesure integer primary key autoincrement unique,
    date varchar,
    id_station integer,
    pressure float,
    wind_heading float,
    wind_speed_avg float,
    wind_speed_max float,
    wind_speed_min float,
    status_state varchar,
    foreign key (id_station) references stations(id_station))''')
                                                     
    db.commit()
    
    cursor.close()
    db.close()
except:
    print("Une erreur est survenue lors de la creation de la base")
    exit(1)
    
print("La base de données a été créée")  

Une erreur est survenue lors de la creation de la base
La base de données a été créée


## Création liste Stations et Mesures

In [3]:
liste_stations = [113, 307, 308]

In [4]:
data_stations=[]
for i in liste_stations:
    q=str(i)
    response = requests.get('http://api.pioupiou.fr/v1/live/'+q)
    response_info = json.loads(response.text)
    data_stations.append([response_info['data']['id'], response_info['data']['meta']['name'], response_info['data']['location']['latitude'], response_info['data']['location']['longitude']])

In [5]:
data_stations

[[113, 'Brétignolles sur mer - La Normandelière', 46.613702, -1.861307],
 [307, 'Pioupiou 307', 46.450683, -1.581863],
 [308, 'Pioupiou 308', 46.692604, -1.765012]]

In [6]:
data_mesures=[]
for i in liste_stations:
    q=str(i)
    response = requests.get('http://api.pioupiou.fr/v1/live/'+q)
    response_info = json.loads(response.text)
    data_mesures.append([response_info['data']['measurements']['date'], response_info['data']['id'], response_info['data']['measurements']['pressure'], response_info['data']['measurements']['wind_heading'], response_info['data']['measurements']['wind_speed_avg'], response_info['data']['measurements']['wind_speed_max'], response_info['data']['measurements']['wind_speed_min'], response_info['data']['status']['state']])

In [7]:
data_mesures

[['2021-03-12T15:00:34.000Z', 113, None, 247.5, 34.5, None, None, 'on'],
 ['2021-03-12T14:55:10.000Z', 307, None, 270, 25, 45, 11.5, 'on'],
 ['2021-03-12T14:49:02.000Z', 308, None, 22.5, 0, 0, 0, 'on']]

## Connexion BDD

In [8]:
conn = sqlite3.connect('foil.db')

## Insertion données

In [9]:
def inserer_donnees(conn, sql_insertion_table, donnees):
    try:
        cursor = conn.cursor()
        for d in donnees:
            cursor.execute(sql_insertion_table, d)
        conn.commit()
    except sqlite3.Error as e:
        print("Erreur lors de l'insertion des données")
        print(e)
        return
    cursor.close()
    print("Les données ont été insérées avec succès")

In [10]:
sql_inserer_station = """
    INSERT INTO stations
    (id_station, name, latitude, longitude)
    VALUES (?, ?, ?, ?)
"""

In [11]:
inserer_donnees(conn, sql_inserer_station, data_stations)

Erreur lors de l'insertion des données
UNIQUE constraint failed: stations.id_station


In [12]:
sql_inserer_mesure = """
    INSERT INTO mesures
    (date, id_station, pressure, wind_heading, wind_speed_avg, wind_speed_max, wind_speed_min, status_state)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""

In [13]:
while True:
    inserer_donnees(conn, sql_inserer_mesure, data_mesures)
    time.sleep(60)

Les données ont été insérées avec succès


In [14]:
last_mesures_113 = '''
SELECT *
FROM mesures
WHERE id_station=113
ORDER BY date DESC
LIMIT 10
'''

last_mesures_113 = pd.read_sql_query(last_mesures_113, conn)
last_mesures_113

,id_mesure,date,id_station,pressure,wind_heading,wind_speed_avg,wind_speed_max,wind_speed_min,status_state
0,439,2021-03-12T15:00:34.000Z,113,None,247.5,34.5,None,None,on
1,394,2021-03-12T13:50:00.000Z,113,None,247.5,30.5,None,None,on
2,397,2021-03-12T13:50:00.000Z,113,None,247.5,30.5,None,None,on
3,400,2021-03-12T13:50:00.000Z,113,None,247.5,30.5,None,None,on
4,403,2021-03-12T13:50:00.000Z,113,None,247.5,30.5,None,None,on
5,406,2021-03-12T13:50:00.000Z,113,None,247.5,30.5,None,None,on
6,409,2021-03-12T13:50:00.000Z,113,None,247.5,30.5,None,None,on
7,412,2021-03-12T13:50:00.000Z,113,None,247.5,30.5,None,None,on
8,415,2021-03-12T13:50:00.000Z,113,None,247.5,30.5,None,None,on
9,418,2021-03-12T13:50:00.000Z,113,None,247.5,30.5,None,None,on


In [15]:
list_last_mesures_113 = last_mesures_113.values.tolist()
list_last_mesures_113

[[439, '2021-03-12T15:00:34.000Z', 113, None, 247.5, 34.5, None, None, 'on'],
 [394, '2021-03-12T13:50:00.000Z', 113, None, 247.5, 30.5, None, None, 'on'],
 [397, '2021-03-12T13:50:00.000Z', 113, None, 247.5, 30.5, None, None, 'on'],
 [400, '2021-03-12T13:50:00.000Z', 113, None, 247.5, 30.5, None, None, 'on'],
 [403, '2021-03-12T13:50:00.000Z', 113, None, 247.5, 30.5, None, None, 'on'],
 [406, '2021-03-12T13:50:00.000Z', 113, None, 247.5, 30.5, None, None, 'on'],
 [409, '2021-03-12T13:50:00.000Z', 113, None, 247.5, 30.5, None, None, 'on'],
 [412, '2021-03-12T13:50:00.000Z', 113, None, 247.5, 30.5, None, None, 'on'],
 [415, '2021-03-12T13:50:00.000Z', 113, None, 247.5, 30.5, None, None, 'on'],
 [418, '2021-03-12T13:50:00.000Z', 113, None, 247.5, 30.5, None, None, 'on']]

In [16]:
last_mesures_307 = '''
SELECT *
FROM mesures
WHERE id_station=307
ORDER BY date DESC
LIMIT 10
'''

last_mesures_307 = pd.read_sql_query(last_mesures_307, conn)

In [17]:
list_last_mesures_307 = last_mesures_307.values.tolist()

In [18]:
last_mesures_308 = '''
SELECT *
FROM mesures
WHERE id_station=308
ORDER BY date DESC
LIMIT 10
'''

last_mesures_308 = pd.read_sql_query(last_mesures_308, conn)

In [19]:
list_last_mesures_308 = last_mesures_308.values.tolist()

In [20]:
file_loader = FileSystemLoader('templates')
env = Environment(loader=file_loader, autoescape=True)

template = env.get_template('index.html')

filename = "html/index.html"
with open(filename, 'w') as dashboard:
    dashboard.write(template.render(data_mesures_113=list_last_mesures_113, data_mesures_307=list_last_mesures_307, data_mesures_308=list_last_mesures_308))